In [1]:
import os

import numpy as np
import pandas as pd
from sqlalchemy import create_engine

db_path = os.path.join(os.path.dirname("__file__"), "..", "..", "data", "ufc.db")
engine = create_engine(f"sqlite:///{db_path}")

In [8]:
query = """
WITH cte1 AS (
    SELECT
        t1.fighter_id,
        t1.'order',
        t1.bout_id,
        t1.bout_id_integer,
        t1.opponent_id,
        CASE
            WHEN t2.overall_percentage IS NOT NULL THEN t2.overall_percentage / 100.0
            ELSE t1.pick_em_percent / 100.0
        END AS community_pick_win_pct,
        (t2.ko_tko_percentage / 100.0) * (t2.overall_percentage / 100.0) AS community_pick_win_by_ko_tko_pct,
        (t2.submission_percentage / 100.0) * (t2.overall_percentage / 100.0) AS community_pick_win_by_submission_pct,
        (t2.decision_percentage / 100.0) * (t2.overall_percentage / 100.0) AS community_pick_win_by_decision_pct
    FROM
        tapology_fighter_histories AS t1
    LEFT JOIN   
        tapology_community_picks AS t2
    ON t1.fighter_id = t2.fighter_id AND t1.bout_id = t2.bout_id
),
cte2 AS (
    SELECT
        fighter_id,
        t1.'order',
        bout_id,
        bout_id_integer,
        opponent_id,
        AVG(community_pick_win_pct) OVER (
            PARTITION BY fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_community_pick_win_pct,
        AVG(community_pick_win_by_ko_tko_pct) OVER (
            PARTITION BY fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_community_pick_win_by_ko_tko_pct,
        AVG(community_pick_win_by_submission_pct) OVER (
            PARTITION BY fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_community_pick_win_by_submission_pct,
        AVG(community_pick_win_by_decision_pct) OVER (
            PARTITION BY fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_community_pick_win_by_decision_pct
    FROM
        cte1 AS t1
),
cte3 AS (
    SELECT
        t1.fighter_id,
        t1.'order',
        t1.bout_id,
        t1.bout_id_integer,
        t1.opponent_id,
        t1.avg_community_pick_win_pct,
        t1.avg_community_pick_win_by_ko_tko_pct,
        t1.avg_community_pick_win_by_submission_pct,
        t1.avg_community_pick_win_by_decision_pct,
        AVG(t2.avg_community_pick_win_pct) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_avg_community_pick_win_pct,
        AVG(t1.avg_community_pick_win_pct - t2.avg_community_pick_win_pct) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_community_pick_win_pct_diff,
        AVG(t2.avg_community_pick_win_by_ko_tko_pct) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_avg_community_pick_win_by_ko_tko_pct,
        AVG(t1.avg_community_pick_win_by_ko_tko_pct - t2.avg_community_pick_win_by_ko_tko_pct) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_community_pick_win_by_ko_tko_pct_diff,
        AVG(t2.avg_community_pick_win_by_submission_pct) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_avg_community_pick_win_by_submission_pct,
        AVG(t1.avg_community_pick_win_by_submission_pct - t2.avg_community_pick_win_by_submission_pct) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_community_pick_win_by_submission_pct_diff,
        AVG(t2.avg_community_pick_win_by_decision_pct) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_avg_community_pick_win_by_decision_pct,
        AVG(t1.avg_community_pick_win_by_decision_pct - t2.avg_community_pick_win_by_decision_pct) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_community_pick_win_by_decision_pct_diff
    FROM
        cte2 AS t1
    LEFT JOIN
        cte2 AS t2
    ON t1.fighter_id = t2.opponent_id AND t1.opponent_id = t2.fighter_id AND t1.bout_id_integer = t2.bout_id_integer
),
cte4 AS (
    SELECT
        t2.ufcstats_id AS fighter_id,
        t3.ufcstats_id AS bout_id,
        t1.avg_community_pick_win_pct,
        t1.avg_community_pick_win_by_ko_tko_pct,
        t1.avg_community_pick_win_by_submission_pct,
        t1.avg_community_pick_win_by_decision_pct,
        t1.avg_opp_avg_community_pick_win_pct,
        t1.avg_community_pick_win_pct_diff,
        t1.avg_opp_avg_community_pick_win_by_ko_tko_pct,
        t1.avg_community_pick_win_by_ko_tko_pct_diff,
        t1.avg_opp_avg_community_pick_win_by_submission_pct,
        t1.avg_community_pick_win_by_submission_pct_diff,
        t1.avg_opp_avg_community_pick_win_by_decision_pct,
        t1.avg_community_pick_win_by_decision_pct_diff
    FROM
        cte3 AS t1
    INNER JOIN
        fighter_mapping AS t2
    ON
        t1.fighter_id = t2.tapology_id
    INNER JOIN
        bout_mapping AS t3
    ON
        t1.bout_id = t3.tapology_id
)
SELECT
    id,
    t2.avg_community_pick_win_pct - t3.avg_community_pick_win_pct AS community_pick_win_pct_diff,
    1.0 * t2.avg_community_pick_win_pct / t3.avg_community_pick_win_pct AS community_pick_win_pct_ratio,
    t2.avg_community_pick_win_by_ko_tko_pct - t3.avg_community_pick_win_by_ko_tko_pct AS community_pick_win_by_ko_tko_pct_diff,
    1.0 * t2.avg_community_pick_win_by_ko_tko_pct / t3.avg_community_pick_win_by_ko_tko_pct AS community_pick_win_by_ko_tko_pct_ratio,
    t2.avg_community_pick_win_by_submission_pct - t3.avg_community_pick_win_by_submission_pct AS community_pick_win_by_submission_pct_diff,
    1.0 * t2.avg_community_pick_win_by_submission_pct / t3.avg_community_pick_win_by_submission_pct AS community_pick_win_by_submission_pct_ratio,
    t2.avg_community_pick_win_by_decision_pct - t3.avg_community_pick_win_by_decision_pct AS community_pick_win_by_decision_pct_diff,
    1.0 * t2.avg_community_pick_win_by_decision_pct / t3.avg_community_pick_win_by_decision_pct AS community_pick_win_by_decision_pct_ratio,
    t2.avg_opp_avg_community_pick_win_pct - t3.avg_opp_avg_community_pick_win_pct AS opp_avg_community_pick_win_pct_diff,
    1.0 * t2.avg_opp_avg_community_pick_win_pct / t3.avg_opp_avg_community_pick_win_pct AS opp_avg_community_pick_win_pct_ratio,
    t2.avg_community_pick_win_pct_diff - t3.avg_community_pick_win_pct_diff AS community_pick_win_pct_diff_diff,
    1.0 * t2.avg_community_pick_win_pct_diff / t3.avg_community_pick_win_pct_diff AS community_pick_win_pct_diff_ratio,
    t2.avg_opp_avg_community_pick_win_by_ko_tko_pct - t3.avg_opp_avg_community_pick_win_by_ko_tko_pct AS opp_avg_community_pick_win_by_ko_tko_pct_diff,
    1.0 * t2.avg_opp_avg_community_pick_win_by_ko_tko_pct / t3.avg_opp_avg_community_pick_win_by_ko_tko_pct AS opp_avg_community_pick_win_by_ko_tko_pct_ratio,
    t2.avg_community_pick_win_by_ko_tko_pct_diff - t3.avg_community_pick_win_by_ko_tko_pct_diff AS community_pick_win_by_ko_tko_pct_diff_diff,
    1.0 * t2.avg_community_pick_win_by_ko_tko_pct_diff / t3.avg_community_pick_win_by_ko_tko_pct_diff AS community_pick_win_by_ko_tko_pct_diff_ratio,
    t2.avg_opp_avg_community_pick_win_by_submission_pct - t3.avg_opp_avg_community_pick_win_by_submission_pct AS opp_avg_community_pick_win_by_submission_pct_diff,
    1.0 * t2.avg_opp_avg_community_pick_win_by_submission_pct / t3.avg_opp_avg_community_pick_win_by_submission_pct AS opp_avg_community_pick_win_by_submission_pct_ratio,
    t2.avg_community_pick_win_by_submission_pct_diff - t3.avg_community_pick_win_by_submission_pct_diff AS community_pick_win_by_submission_pct_diff_diff,
    1.0 * t2.avg_community_pick_win_by_submission_pct_diff / t3.avg_community_pick_win_by_submission_pct_diff AS community_pick_win_by_submission_pct_diff_ratio,
    t2.avg_opp_avg_community_pick_win_by_decision_pct - t3.avg_opp_avg_community_pick_win_by_decision_pct AS opp_avg_community_pick_win_by_decision_pct_diff,
    1.0 * t2.avg_opp_avg_community_pick_win_by_decision_pct / t3.avg_opp_avg_community_pick_win_by_decision_pct AS opp_avg_community_pick_win_by_decision_pct_ratio,
    t2.avg_community_pick_win_by_decision_pct_diff - t3.avg_community_pick_win_by_decision_pct_diff AS community_pick_win_by_decision_pct_diff_diff,
    1.0 * t2.avg_community_pick_win_by_decision_pct_diff / t3.avg_community_pick_win_by_decision_pct_diff AS community_pick_win_by_decision_pct_diff_ratio,
    CASE
        WHEN red_outcome = 'W' THEN 1
        ELSE 0
    END AS red_win
FROM ufcstats_bouts AS t1
LEFT JOIN cte4 AS t2
ON t1.id = t2.bout_id AND t1.red_fighter_id = t2.fighter_id
LEFT JOIN cte4 AS t3
ON t1.id = t3.bout_id AND t1.blue_fighter_id = t3.fighter_id
WHERE event_id IN (
    SELECT id FROM ufcstats_events
    WHERE is_ufc_event = 1 AND date >= '2008-04-19' AND date < '2021-01-01'
) AND red_outcome IN ('W', 'L') AND outcome_method != 'DQ'
"""

df = pd.read_sql(query, engine)
df

,id,community_pick_win_pct_diff,community_pick_win_pct_ratio,community_pick_win_by_ko_tko_pct_diff,community_pick_win_by_ko_tko_pct_ratio,community_pick_win_by_submission_pct_diff,community_pick_win_by_submission_pct_ratio,community_pick_win_by_decision_pct_diff,community_pick_win_by_decision_pct_ratio,opp_avg_community_pick_win_pct_diff,...,community_pick_win_by_ko_tko_pct_diff_ratio,opp_avg_community_pick_win_by_submission_pct_diff,opp_avg_community_pick_win_by_submission_pct_ratio,community_pick_win_by_submission_pct_diff_diff,community_pick_win_by_submission_pct_diff_ratio,opp_avg_community_pick_win_by_decision_pct_diff,opp_avg_community_pick_win_by_decision_pct_ratio,community_pick_win_by_decision_pct_diff_diff,community_pick_win_by_decision_pct_diff_ratio,red_win
0,be38ed9ccfe2ee03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,eb1b371dfc37fcdb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,219bd976b8ca745d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,af178adff964d854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,920194911d727a38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4907,cd150cd28738a7c5,-0.043000,0.936107,-0.313138,0.467145,0.045948,17.972823,0.176786,4.112738,-0.093514,...,0.417491,0.058470,3.599154,0.047900,-1.166163,0.011951,1.137289,0.140556,-0.958170,1
4908,8955ea3c7c332e6c,0.073864,1.125725,0.088012,1.409963,0.001806,1.099804,0.009403,1.035570,-0.005559,...,-0.980367,0.015985,1.218199,0.017805,0.686620,0.074042,1.235795,0.095707,0.021922,0
4909,3d35eb2d46bf74de,-0.219048,0.719170,0.365047,3.091364,0.011702,3.461094,0.018292,1.258752,-0.058022,...,-0.306608,0.006811,1.239954,-0.003201,1.472595,0.049461,1.249507,-0.155808,-30.047679,1
4910,014f1da2083ca174,0.036471,1.061569,0.206770,2.474372,-0.158800,0.132809,-0.024187,0.910159,0.127854,...,-4.581354,0.019401,1.481961,-0.199513,-0.037095,0.110340,1.453327,-0.147776,-37.706791,1
